# 0.0 Imports & Functions

In [31]:
import sqlite3
import pandas.io.sql as psql

In [32]:
# Function to connect to the database.
def connect_db(base = 'db_olist.sqlite'):
    # To connect to the SQLite database (the file will be created if it doesn't exist)
    return sqlite3.connect(base)

def execute_query_db(query, cursor):
    # Drop the temporary table if it exists.
    cursor.execute(query)


## 0.1 Database connection and execution of auxiliary queries.

In [33]:
# Connet to database
conn = connect_db();

# Create a cursor
cursor = conn.cursor();

# Execute DROP TABLE
execute_query_db("DROP TABLE IF EXISTS geolocation_temporary;", cursor);

# Execute Creation of the temporary table.
execute_query_db("""
    CREATE TEMPORARY TABLE geolocation_temporary (
        geolocation_zip_code_prefix INTEGER,
        count INTEGER,
        min_lat REAL,
        media_lat REAL,
        max_lat REAL,
        random_num REAL,
        min_lng REAL,
        media_lng REAL,
        max_lng REAL
    );
""", cursor);

# Insertion of data into the temporary table
execute_query_db("""
    INSERT INTO geolocation_temporary
    SELECT
        geolocation_zip_code_prefix,
        COUNT(geolocation_zip_code_prefix),
        MIN(geolocation_lat) AS min_lat,
        AVG(geolocation_lat) AS media_lat,
        MAX(geolocation_lat) AS max_lat,
        ABS(RANDOM())/10000000000000000000 AS random_num,
        MIN(geolocation_lng) AS min_lng,
        AVG(geolocation_lng) AS media_lng,
        MAX(geolocation_lng) AS max_lng
    FROM geolocation
    GROUP BY geolocation_zip_code_prefix
    ORDER BY geolocation_zip_code_prefix;
""", cursor);

## 0.2 Main query

In [34]:
# Main query
query = """
    SELECT
        ROW_NUMBER() OVER () AS row_number,
        ABS(RANDOM())/10000000000000000000 AS random_num ,
        o.order_id ,
        o.customer_id ,
        o.order_status ,
        o.order_approved_at  ,
        o.order_delivered_customer_date  ,
        o.order_estimated_delivery_date ,
        oi.product_id ,
        oi.seller_id ,
        c.customer_zip_code_prefix ,
        s.seller_zip_code_prefix ,
        gc.media_lat AS MEDIA_LAT_CUSTOMER,
        gc.media_lng AS MEDIA_LNG_CUSTOMER,
        gs.media_lat AS MEDIA_LAT_SELLER,
        gs.media_lng AS MEDIA_LNG_SELLER,
        SQRT(POW(gc.media_lat - gs.media_lat, 2) + POW(gc.media_lng - gs.media_lng, 2)) AS dist_degrees,
        p.product_weight_g ,
        (p.product_length_cm * p.product_height_cm * p.product_width_cm) AS product_volume_cm2 ,
        p.product_category_name,
        julianday(o.order_delivered_customer_date) - julianday(o.order_approved_at) AS delta_time_real,
        julianday(o.order_estimated_delivery_date) - julianday(o.order_approved_at) AS delta_time_dataset,
        julianday(o.order_estimated_delivery_date) - julianday(o.order_delivered_customer_date) AS error_model_dataset
    FROM orders o 
    INNER JOIN order_items oi ON o.order_id = oi.order_id
    INNER JOIN customer c ON c.customer_id = o.customer_id 
    INNER JOIN sellers s ON oi.seller_id = s.seller_id 
    INNER JOIN geolocation_temporary gc ON gc.geolocation_zip_code_prefix = c.customer_zip_code_prefix
    INNER JOIN geolocation_temporary gs ON gs.geolocation_zip_code_prefix = s.seller_zip_code_prefix
    INNER JOIN products p ON p.product_id = oi.product_id
    WHERE 1=1
    AND o.order_status = 'delivered'
    AND o.order_approved_at is NOT NULL
    AND o.order_delivered_customer_date is not null
    AND o.order_estimated_delivery_date  is not null
    AND gs.media_lat is not null
    AND gs.media_lng is not null
    AND gc.media_lat is not null
    AND gc.media_lng is not null
    AND p.product_category_name IS NOT NULL
    AND p.product_weight_g <> 0
    AND delta_time_real > 0;

"""
# Return selection in dataframe format
df = psql.read_sql( query, conn )

# Close cursor and connect
cursor.close()
conn.close()